# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 63 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.12101


extracting tarball to tmp_2303.12101...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.12101/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2303.12579...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.12598


extracting tarball to tmp_2303.12598...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.12598/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'appendices' from 'tmp_2303.12598/sections/appendices.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'sections/acknowledgements' from 'tmp_2303.12598/sections/acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12101-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12101) | **Stellar associations powering HII regions $\unicode{x2013}$ I. Defining  an evolutionary sequence**  |
|| Fabian Scheuermann, et al. -- incl., <mark>Kathryn Kreckel</mark>, <mark>Stephen Hannon</mark>, <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *15 pages, 12 figures. Accepted for publication in MNRAS*|
|**Abstract**| Connecting the gas in HII regions to the underlying source of the ionizing radiation can help us constrain the physical processes of stellar feedback and how HII regions evolve over time. With PHANGS$\unicode{x2013}$MUSE we detect nearly 24,000 HII regions across 19 galaxies and measure the physical properties of the ionized gas (e.g. metallicity, ionization parameter, density). We use catalogues of multi-scale stellar associations from PHANGS$\unicode{x2013}$HST to obtain constraints on the age of the ionizing sources. We construct a matched catalogue of 4,177 HII regions that are clearly linked to a single ionizing association. A weak anti-correlation is observed between the association ages and the H$\alpha$ equivalent width EW(H$\alpha$), the H$\alpha$/FUV flux ratio and the ionization parameter, log q. As all three are expected to decrease as the stellar population ages, this could indicate that we observe an evolutionary sequence. This interpretation is further supported by correlations between all three properties. Interpreting these as evolutionary tracers, we find younger nebulae to be more attenuated by dust and closer to giant molecular clouds, in line with recent models of feedback-regulated star formation. We also observe strong correlations with the local metallicity variations and all three proposed age tracers, suggestive of star formation preferentially occurring in locations of locally enhanced metallicity. Overall, EW(H$\alpha$) and log q show the most consistent trends and appear to be most reliable tracers for the age of an HII region. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12110-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12110) | **The Gas Mass Reservoir of Quiescent Galaxies at Cosmic Noon**  |
|| David Blánquez-Sesé, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *Accepted to A&A on the 16/03/2023*|
|**Abstract**| We present a 1.1mm stacking analysis of moderately massive (log($M_{*}$/$M_{\odot}$) = 10.7 $\pm$ 0.2) quiescent galaxies (QGs) at $\langle z\rangle \sim1.5$, searching for cold dust continuum emission, an excellent tracer of dust and gas mass. Using both the recent GOODS-ALMA survey as well as the full suite of ALMA Band-6 ancillary data in the GOODS-S field, we report the tentative detection of dust continuum equivalent of dust mass log($M_{dust}$/$M_{\odot}$) = 7.47 $\pm$ 0.13 and gas mass log($M_{gas}$/$M_{\odot}$) = 9.42 $\pm$ 0.14. The emerging gas fraction is $f_{gas}$ = 5.3 $\pm$ 1.8%, consistent with the results of previous stacking analyses based on lower resolution sub(mm) observations. Our results support the scenario where high-z QGs have an order of magnitude larger $f_{gas}$ compared to their local counterparts and have experienced quenching with a non negligible gas reservoir in their interstellar medium - i.e. with gas retention. Subsequent analysis yields an anti-correlation between the $f_{gas}$ and the stellar mass of QGs, especially in the high mass end where galaxies reside in the most massive haloes. The $f_{gas}$ - $M_{*}$ anti-correlation promotes the selection bias as a possible solution to the tension between the stacking results pointing towards gas retention in high-z QGs of moderate $M_{*}$ and the studies of individual targets that favour a fully depleted ISM in massive (log($M_{*}$/$M_{\odot}$) high-z QGs. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12598-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12598) | **A LOFAR sample of luminous compact sources coincident with nearby dwarf  galaxies**  |
|| D. Vohl, et al. -- incl., <mark>C. Zhang</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *Submitted. arXiv admin note: text overlap with arXiv:2303.11967*|
|**Abstract**| The vast majority of extragalactic, compact continuum radio sources are associated with star formation or jets from (super)massive black holes and, as such, are more likely to be found in association with starburst galaxies or early type galaxies. Recently, two new populations of radio sources have been identified: (a) compact and persistent sources (PRS) associated with fast radio bursts (FRB) in dwarf galaxies and (b) compact sources in dwarf galaxies that could belong to the long-sought population of intermediate-mass black holes. Despite the interesting aspects of these newly found sources, the current sample size is small, limiting scrutiny of the underlying population. Here, we present a search for compact radio sources coincident with dwarf galaxies. We search the LOFAR Two-meter Sky Survey (LoTSS) -- the most sensitive low-frequency (144 MHz central frequency) large-area survey for optically thin synchrotron emission to date. Exploiting LoTSS' high spatial resolution (6 arcsec) and low astrometric uncertainty (about 0.2 arcsec), we match its compact sources to the compiled sample of dwarf galaxies in the Census of the Local Universe -- an H{\alpha} survey with the Palomar Observatory's 48-inch Samuel Oschin Telescope. We identify 29 overluminous compact radio sources, evaluate the probability of chance alignment within the sample, investigate the potential nature of these sources, and evaluate their volumetric density. While optical line-ratio diagnostics on the nebular lines from the host galaxies prefer a star-formation origin (against an AGN origin), future high angular resolution radio data is necessary to ascertain the origin of the radio sources. We discuss planned strategies to differentiate them between candidate FRB hosts and intermediate-mass black holes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12542-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12542) | **Discovery of the linear energy-dependence of the spectral lag of X-ray  bursts from SGR J1935+2154**  |
|| Shuo Xiao, et al. -- incl., <mark>You-Li Tuo</mark>, <mark>Shuang-Nan Zhang</mark>, <mark>Yan-Qiu Zhang</mark>, <mark>He Gao</mark>, <mark>Cheng-Kui Li</mark>, <mark>Xiao-Bo Li</mark>, <mark>Xin-Qiao Li</mark>, <mark>Li-Ming Song</mark>, <mark>Fan Zhang</mark>, <mark>Wei Xie</mark>, <mark>Ti-Pei Li</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *accepted for publication in MNRAS*|
|**Abstract**| Spectral lag of the low-energy photons with respect to the high-energy ones is a common astrophysical phenomenon (such as Gamma-ray bursts and the Crab pulsar) and may serve as a key probe to the underlying radiation mechanism. However, spectral lag in keV range of the magnetar bursts has not been systematically studied yet. In this work, we perform a detailed spectral lag analysis with the Li-CCF method for SGR J1935+2154 bursts observed by {\it Insight}-HXMT, GECAM and Fermi/GBM from July 2014 to Jan 2022. We discover that the spectral lags of about 61\% (non-zero significance >1$\sigma$) bursts from SGR J1935+2154 are linearly dependent on the photon energy ($E$) with $t_{\rm lag}(E)=\alpha (E/{\rm keV})+C$, which may be explained by a linear change of the temperature of the blackbody-emitting plasma with time. The distribution of the slope ($\alpha$) approximately follows a Gaussian function with mean and standard deviation of 0.02 ms/keV (i.e. high-energy photons arrive earlier) and 0.02 ms/keV, respectively. We also find that the distribution can be well fitted with three Gaussians with mean values of $\sim$ -0.009, 0.013 and 0.039 ms/keV, which may correspond to different origins of the bursts. These spectral lag features may have important implications on the magnetar bursts. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12577-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12577) | **Detection of Dust in High-Velocity Cloud Complex C -- Enriched Gas  Accreting onto the Milky Way**  |
|| Andrew J. Fox, et al. -- incl., <mark>Philipp Richter</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *8 pages, 3 figures, accepted for publication in ApJ Letters*|
|**Abstract**| We present the first detection of dust depletion in Complex C, a massive, infalling, low-metallicity high-velocity cloud in the northern Galactic hemisphere that traces the ongoing accretion of gas onto the Milky Way. We analyze a very high signal-to-noise HST/COS spectrum of AGN Mrk 817 formed by coadding 165 individual exposures taken under the AGN STORM 2 program, allowing us to determine dust-depletion patterns in Complex C at unprecedented precision. By fitting Voigt components to the O I, S II, N I, Si II, Fe II, and Al II absorption and applying ionization corrections from customized Cloudy photoionization models, we find sub-solar elemental abundance ratios of [Fe/S]=-0.42+/-0.08, [Si/S]=-0.29+/-0.05, and [Al/S]=-0.53+/-0.08. These ratios indicate the depletion of Fe, Si, and Al into dust grains, since S is mostly undepleted. The detection of dust provides an important constraint on the origin of Complex C, as dust grains indicate the gas has been processed through galaxies, rather than being purely extragalactic. We also derive a low metallicity of Complex C of [S/H]=-0.51+/-0.16 (31% solar), confirming earlier results from this sightline. We discuss origin models that could explain the presence of dust in Complex C, including Galactic fountain models, tidal stripping from the Magellanic Clouds or other satellite galaxies, and precipitation of coronal gas onto dust-bearing ``seed" clouds. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12579-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12579) | **Mixing of materials in magnetised core-collapse supernova remnants**  |
|| <mark>Meyer D. M.-A.</mark>, Pohl M., Petrov M., Egberts K. |
|*Appeared on*| *2023-03-23*|
|*Comments*| *Accepted at MNRAS*|
|**Abstract**| Core-collapse supernova remnants are structures of the interstellar medium (ISM) left behind the explosive death of most massive stars (smaller or equal to 40 Mo). Since they result in the expansion of the supernova shock wave into the gaseous environment shaped by the star wind history, their morphology constitutes an insight into the past evolution of their progenitor star. Particularly, fast-moving massive stars can produce asymmetric core-collapse supernova remnants. We investigate the mixing of materials in core-collapse supernova remnants generated by a moving massive 35 Mo star, in a magnetised ISM. Stellar rotation and the wind magnetic field are time-dependently included into the models which follow the entire evolution of the stellar surroundings from the zero age main sequence to 80 kyr after the supernova explosion. It is found that very little main sequence material is present in remnants from moving stars, that the Wolf-Rayet wind mixes very efficiently within the 10 kyr after the explosion, while the red supergiant material is still unmixed by 30 per cent within 50 kyr after the supernova. Our results indicate that the faster the stellar motion, the more complex the internal organisation of the supernova remnant and the more effective the mixing of ejecta therein. In contrast, the mixing of stellar wind material is only weakly affected by progenitor motion, if at all. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12163-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12163) | **New constraints on the presence of debris disks around G 196-3 B and VHS  J125601.92-125723.9 b**  |
|| <mark>O. V. Zakhozhay</mark>, et al. -- incl., <mark>D. A. Semenov</mark>, <mark>M. Perez-Torres</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| *Accepted fro publication in A&A*|
|**Abstract**| We obtained deep images of G 196-3 B and VHS J1256-1257 b with the NOrthern Extended Millimeter Array (NOEMA) at 1.3 mm. These data were combined with recently published Atacama Large Millimeter Array (ALMA) and Very Large Array (VLA) data of VHS J1256-1257 b at 0.87 mm and 0.9 cm, respectively. Neither G 196-3 B nor VHS J1256-1257 b were detected in the NOEMA, ALMA and VLA data. At 1.3 mm, we imposed flux upper limits of 0.108 mJy (G 196-3 B) and 0.153 mJy (VHS J1256-1257 b) with a 3-sigma confidence. Using the flux upper limits at the millimeter and radio wavelength regimes, we derived maximum values of 0.016 M$_{\rm Earth}$ and 0.004 M$_{\rm Earth}$ for the mass of any cold dust that might be surrounding G 196-3 B and VHS J1256-1257 b, respectively. We put our results in the context of other deep millimeter observations of free-floating and companion objects with substellar masses smaller than 20 M$_{\rm Jupiter}$ and ages between 1 and a few hundred million years. Only two very young objects are detected out of a few tens concluding, as other groups did before, that the disks around these very low-mass objects must have small masses and possibly reduced sizes. If debris disks around substellar objects scale down in a similar manner as protoplanetary disks do, millimeter observations of moderately young brown dwarfs and planets must be at least two orders of magnitude deeper for being able to detect and characterize their surrounding debris disks. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12166-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12166) | **N-body simulation of binary star mass transfer using NVDIA GPUs**  |
|| Shaukat Goderya, et al. -- incl., <mark>Edward Smith</mark> |
|*Appeared on*| *2023-03-23*|
|*Comments*| **|
|**Abstract**| Binary star systems are of particular interest to astronomers because they can be used as astrophysical laboratories to study the properties and processes of stars. Between 70% to 90% of the stars in our galaxy are part of a binary star system. Among the many types of binary systems observed, the dynamics of semi-detached and contact systems are the most interesting because they exhibit mass transfer, which changes the composition and life cycle of both stars. The time scales of the mass transfer process are extremely large which makes the process impossible to capture through physical observation. Computer simulations have proved invaluable in refining our understanding of the mass transfer processes. Here we introduce an intuitive, computationally efficient, gravity centered model that simulates the filling of the Roche lobe of an expanding star and its transfer of mass through the first Lagrangian point. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.12101.md
exported in  _build/html/2303.12110.md
    + _build/html/tmp_2303.12110/./Figures/f_gas_final.png
    + _build/html/tmp_2303.12110/./Figures/fgas_mstar_15.0.png
    + _build/html/tmp_2303.12110/./Figures/Galaxy_selection_2.png
    + _build/html/tmp_2303.12110/./Figures/UVJ_diagram.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uncertainty}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\StoN}{\mathrm{S}/\mathrm{N}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\change}[1]{{\color{orange}#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uncertainty}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\StoN}{\mathrm{S}/\mathrm{N}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\change}[1]{{\color{orange}#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Stellar associations powering $\HII$ regions -- I. Defining an evolutionary sequence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.12101-b31b1b.svg)](https://arxiv.org/abs/2303.12101)<mark>Appeared on: 2023-03-23</mark> - _15 pages, 12 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

Fabian Scheuermann, et al. -- incl., <mark><mark>Kathryn Kreckel</mark></mark>, <mark><mark>Stephen Hannon</mark></mark>, <mark><mark>Janice C. Lee</mark></mark>, <mark><mark>Frank Bigiel</mark></mark>, <mark><mark>Eva Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Connecting the gas in $\HII$ regions to the underlying source of the ionizing radiation can help us constrain the physical processes of stellar feedback and how $\HII$ regions evolve over time.With PHANGS--MUSE we detect nearly $\num{24000}$ $\HII$ regions across 19 galaxies and measure the physical properties of the ionized gas (e.g. metallicity, ionization parameter, density).We use catalogues of multi-scale stellar associations from PHANGS-- $_HST_$ to obtain constraints on the age of the ionizing sources.We construct a matched catalogue of $\num{4177}$ $\HII$ regions that are clearly linked to a single ionizing association.A weak anti-correlation is observed between the association ages and the $\HA$ equivalent width $\EW$ , the $\HA/\FUV$ flux ratio and the ionization parameter, $\log q$ .As all three are expected to decrease as the stellar population ages, this could indicate that we observe an evolutionary sequence.This interpretation is further supported by correlations between all three properties.Interpreting these as evolutionary tracers, we find younger nebulae to be more attenuated by dust and closer to giant molecular clouds, in line with recent models of feedback-regulated star formation.We also observe strong correlations with the local metallicity variations and all three proposed age tracers, suggestive of star formation preferentially occurring in locations of locally enhanced metallicity.Overall, $\EW$ and $\log q$ show the most consistent trends and appear to be most reliable tracers for the age of an $\HII$ region.

</div>



<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdust}{M_{\rm dust}}$
$\newcommand{\Mstar}{M_{\rm \ast}}$
$\newcommand{\Msol}{\rm M_{\rm \odot}}$
$\newcommand{\fgas}{f_{\rm gas}}$
$\newcommand{\fdust}{f_{\rm dust}}$
$\newcommand{\Gobat}{{\color{blue} G18}}$
$\newcommand{\Magdis}{{\color{blue} M21}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Mgas}{M_{\rm gas}}$
$\newcommand{\Mdust}{M_{\rm dust}}$
$\newcommand{\Mstar}{M_{\rm \ast}}$
$\newcommand{\Msol}{\rm M_{\rm \odot}}$
$\newcommand{\fgas}{f_{\rm gas}}$
$\newcommand{\fdust}{f_{\rm dust}}$
$\newcommand{\Gobat}{{\color{blue} G18}}$
$\newcommand{\Magdis}{{\color{blue} M21}}$</div>



<div id="title">

# The gas mass reservoir of quiescent galaxies at cosmic noon

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.12110-b31b1b.svg)](https://arxiv.org/abs/2303.12110)<mark>Appeared on: 2023-03-23</mark> - _Accepted to A&A on the 16/03/2023_

</div>
<div id="authors">

David Blánquez-Sesé, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present a 1.1mm stacking analysis of moderately massive (log( $\Mstar$ / $\Msol$ ) = 10.7 $\pm$ 0.2) quiescent galaxies (QGs) at $\langle z\rangle \sim1.5$ , searching for cold dust continuum emission, an excellent tracer of dust and gas mass. Using both the recent GOODS-ALMA survey as well as the full suite of ALMA Band-6 ancillary data in the GOODS-S field, we report the tentative detection of dust continuum equivalent of   dust mass log( $\Mdust$ / $\Msol$ ) = 7.47 $\pm$ 0.13 and gas mass log( $\Mgas$ / $\Msol$ ) = 9.42 $\pm$ 0.14. The emerging gas fraction is $\fgas$ = 5.3 $\pm$ 1.8 \% , consistent with the results of previous stacking analyses based on lower resolution sub(mm) observations. Our results support the scenario where high $-z$ QGs have an order of magnitude larger $\fgas$ compared to their local counterparts and have experienced quenching with a non negligible gas reservoir in their interstellar medium - i.e. with gas retention. Subsequent analysis yields an anti-correlation between the $\fgas$ and the stellar mass of QGs, especially in the high mass end where galaxies reside in the most massive haloes. The $\fgas$ - $\Mstar$ anti-correlation promotes the selection bias as a possible solution to the tension between the stacking results pointing towards gas retention in high $-z$ QGs of moderate $\Mstar$ and the studies of individual targets that favour a fully depleted ISM in massive (log( $\Mstar$ / $\Msol$ ) > 11.2) high $-z$ QGs.

</div>

<div id="div_fig1">

<img src="tmp_2303.12110/./Figures/f_gas_final.png" alt="Fig6.1" width="50%"/><img src="tmp_2303.12110/./Figures/fgas_mstar_15.0.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** **Gas and dust fractions of QGs**_Top_: Selection of $\fdust$ and $\fgas$ measurements as a function of  redshift for QGs. Circles correspond to dust derived gas fractions: this work and  previous stacks studies ($\Gobat$ and $\Magdis$) are shown in red and blue, respectively. The white circles show two different estimates (connected by a grey dotted line) for a sample of individually observed lensed galaxies. The lower values correspond to those presented in  ([Whitaker, Williams and Mowla (2021)]())  and the upper values show the new estimates provided in  ([Gobat and Liu (2022)]()) . The grey diamonds represent CO derived $\fgas$ estimates  ([Sargent, Daddi and Bournaud 2015](), [Bezanson, Spilker and Williams 2019](), [Williams, Spilker and Whitaker 2021]()) . The red dashed area embeds $\fgas$ measurements of local QGs obtained for the ATLAS3D sample  ([Young, Bureau and Davis 2011](), [Cappellari, McDermid and Alatalo 2013](), [Davis, Young and Crocker 2014]()) . The blue shaded area and the purple dashed line represent the best fit to the $\Magdis$ data and the  ([Gobat, Magdis and Valentino (2020)]())  model respectively. For reference, we add the $\fgas$ evolution of main sequence galaxies according to  ([Liu, Lang and Magnelli (2019)]()) . _Bottom_: Dust and gas fraction as a function of stellar mass for measurements at $z \sim 1.5$. The symbols are the same as in the top panel. The dotted line shows the $\fgas$ prediction according to the  ([Davé, Finlator and Oppenheimer (2012)]())  galaxy evolution models, color coded as a function of $M_{\mathrm{halo}}$. For reference, we add the $\fgas$ - $\Mstar$ trend measured by  ([Magdis, Daddi and Béthermin (2012)](), [Liu, Lang and Magnelli (2019)]()) . The light yellow scattered diamonds and arrows mark the $\fgas$ detections and upper limits for local QGs with the corresponding best fit plotted as a black dashed line. (*fig:f_gas_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2303.12110/./Figures/Galaxy_selection_2.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Galaxy selection.** Density plot of the used ZFOURGE catalogue in the redshift vs stellar mass plane, consisting of a total 13299 galaxies. The red shaded area represents the region covered by our redshift and stellar mass selection criteria ($1 < z < 3$ and 10.20 $<$ log($\Mstar$/$\Msol$) $<$ 11.50) embedding 852 sources. The orange circles correspond to the QGs that constitute our final selection.  (*fig:galaxy_selection*)

</div>
<div id="div_fig3">

<img src="tmp_2303.12110/./Figures/UVJ_diagram.png" alt="Fig2" width="100%"/>

**Figure 2. -** **UVJ colour diagram.** Distribution of the parent sample of 435 galaxies that meet our selection criteria in the $U-V$, $V-J$ colour-colour space, colour coded by their log(SFR). The red box represents the quiescent region limits defined in  ([Schreiber, Pannella and Elbaz (2015)]()) , which enclose the 140 QGs from which we draw our final sample. (*fig:UVJ_diagram*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

203  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
